In [1]:
%run forward_kinematics.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4
Task Space Box:
{'x_min': -0.05772180470827806, 'x_max': 0.03015406093747365, 'y_min': -0.06853860224338981, 'y_max': 0.057466988088834825, 'z_min': -0.2216507997959984, 'z_max': 0.3100041333006747}


In [2]:
J = dIFF_KinJ4
J_plus = cs.pinv(J)
I = cs.SX.eye(J.size2())

J_ned = cs.SX.zeros((10, 10))
J_ned[:6,:6] = J_uv
J_ned[6:,6:] = cs.SX.eye(4)

J_ned_inv = cs.inv(J_ned)

In [3]:
# --- Secondary Task: joint limit cost---
# stay away from the joint limits 

sigma_wq_i = 0
for i in range(arm_ss.n.size1()):
    q_i_bar = (arm_ss.uvms_ul[i] + arm_ss.uvms_ll[i])/2
    wq_i = ((arm_ss.n[i] - q_i_bar)/(arm_ss.uvms_ul[i] - arm_ss.uvms_ll[i]))**2
    sigma_wq_i = sigma_wq_i + wq_i
    
    
wq =  (1/(2*arm_ss.n.size1()))*sigma_wq_i

J_wq = cs.jacobian(wq, arm_ss.n)

# qdot_sec = cs.diag(uvms_ss.k0)@J_wq.T

In [4]:
# --- Secondary Task: Collision-avoidance cost ---
# Suppose you have a function that calculates w_col(q)
# w_col = collision_cost(q, base_geometry, manipulator_geometry)
# J_col = jacobian(w_col, q)

In [5]:
# Weighted combination of the two secondary tasks
J_total = J_wq
# Secondary velocity
qdot_sec = cs.diag(arm_ss.k0) @ J_total.T



In [6]:
# task projection
IK_v = J_ned_inv@(J_plus@arm_ss.des_v + (I - J_plus@J)@qdot_sec)
IK_v_func = cs.Function('d_iK', [arm_ss.des_v, arm_ss.n, arm_ss.uvms_ul, arm_ss.uvms_ll, arm_ss.k0, arm_ss.base_T], [IK_v])
IK_v_func.save('diff_iK.casadi')

In [7]:
# differential inverse kinematics
# IK_v

In [8]:
IK_v_func([0,0,0, 0,0,0], [0,0,0, 0.01,0.01,0.01 ,1,1,1,1], [10, 10, 10, cs.pi,cs.pi,cs.pi, 2*cs.pi, 2*cs.pi, 2*cs.pi, 2*cs.pi], [-10, -10, -10, -cs.pi,-cs.pi,-cs.pi, 0,0,0,0], [1,1,1, 1,1,1, 1,1,1,1], alpha.base_T0)

DM([0.000700213, 0.000725986, -0.000772064, 3.13955e-05, 2.01581e-05, 2.42778e-05, -0.00532236, -0.00534529, -0.0053826, -0.00534664])